In [106]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import os, re

In [107]:
OUTPUT_EXCEL = '월별원내약품사용현황.xlsx'

In [108]:
# 데이타셋 준비
data_source_dir = '사용량월별통계/원내'
dfs = []
for fname in os.listdir(data_source_dir):
    fn, ext = os.path.splitext(fname)
    if ext in ['.xls', '.xlsx']:
        df = pd.read_excel(os.path.join(data_source_dir, fname))
        df['사용(개시)년월'] = fn
        dfs.append(df)
use_amount_df = pd.concat(dfs, ignore_index=True)


In [109]:
drug_standard_df = pd.read_json('drug.json').T

drug_info_df = pd.read_excel('약품정보.xls')

use_amount_df = pd.merge(drug_info_df, use_amount_df[['사용량', '약품코드', '사용(개시)년월']], on='약품코드', how='left')

use_amount_df = pd.merge(use_amount_df, drug_standard_df[['보험코드', '제품명', '판매사', '성분/함량']], left_on='EDI코드', right_on='보험코드', how='left')

use_amount_df['제품명'] = use_amount_df['제품명'].fillna(use_amount_df['약품명(한글)'])

use_amount_df['사용개시년월'] = use_amount_df['수가시작일자'].map(lambda x: str(x)[0:4]+'-'+str(x)[4:6])

use_amount_df['사용(개시)년월'] = use_amount_df['사용(개시)년월'].fillna(use_amount_df['사용개시년월'])
use_amount_df['성분명'] = use_amount_df['성분명'].fillna(use_amount_df['성분/함량'])

use_amount_df['원내/원외 처방구분'] = use_amount_df['원내/원외 처방구분'].map({1: '원외', 2: '원외/원내', 3: '원내'})
use_amount_df['약품법적구분'] = use_amount_df['약품법적구분'].map({0: '일반', 1: '마약', 2: '향정약', 3: '독약', 4: '한방약', 5: '고가약'})

In [110]:
def get_last(s):
    try:
        return max(s)
    except:
        return s

In [111]:
months = use_amount_df['사용(개시)년월'].unique()
months = sorted(months.tolist(), reverse=1)
use_amount_df['최후사용월'] = use_amount_df.groupby(['제품명'])['사용(개시)년월'].transform(get_last)
use_amount_df['최근미사용월수'] = use_amount_df['최후사용월'].map(lambda x: months.index(x) if x in months else -1)

In [112]:
use_amount_in_df = use_amount_df[use_amount_df['원내/원외 처방구분'] != '원외']

In [113]:
# 작년 신약만
new_drugs = pd.read_excel('2016신약심의결과.xlsx')
new_drugs = new_drugs[['신청과', '신청과장', '구분', '약품코드', '보험코드']]
use_amount_in_df = pd.merge(use_amount_in_df, new_drugs, on='약품코드' )

In [114]:
use_amount_in_df['사용량'] = use_amount_in_df['사용량'].fillna('오픈후미사용')

In [115]:
pat = '(\(([^\d].*?)\)+\s*)|퇴장방지\s*|생산원가보전,*\s*|사용장려(비\s*\d+원|및|비용지급,*\s*)'
use_amount_in_df = use_amount_in_df.rename(columns={'제품명': '약품명(드럭인포)', '약품명(한글)': '약품명(원내)'})
use_amount_in_df['약품명(드럭인포)'] = use_amount_in_df['약품명(드럭인포)'].str.replace(pat, '')

In [116]:
pvt = use_amount_in_df.pivot_table(index = ['EDI코드','약품명(드럭인포)', '성분명','약품코드','약품명(원내)','효능코드명','규격단위', '최근미사용월수', '신청과장'], columns=['사용(개시)년월'], values=['사용량'], aggfunc=sum)

In [117]:
pvt.to_excel(OUTPUT_EXCEL)
os.startfile(OUTPUT_EXCEL)

In [16]:
use_amount_in_df

,약품코드,약품명(영문),약품명(원내),성분명,시작일자,종료일자,원내/원외 처방구분,보험단가,일반단가,효능코드(보건복지부),...,고주의약품분류,사용량,사용(개시)년월,보험코드,약품명(드럭인포),판매사,성분/함량,사용개시년월,최후사용월,최근미사용월수
0,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,19,2016-09,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
1,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,365,2016-10,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
2,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,240,2016-11,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
3,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,349,2016-12,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
4,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,275,2017-01,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
5,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,235,2017-02,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
6,10DW1L,10% D/W 1000ml (씨제이),10DW1000 10% D/W 1000ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1735.0,3470.0,323.0,...,NaN,300,2017-03,640001131,씨제이10%포도당주사액 1000mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
7,10DW500,10DW500 10%포도당주사액 500ml (씨제이),10DW500 10%포도당주사액 500ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1393.0,2780.0,323.0,...,NaN,12,2016-09,640001121,씨제이10%포도당주사액 500mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
8,10DW500,10DW500 10%포도당주사액 500ml (씨제이),10DW500 10%포도당주사액 500ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1393.0,2780.0,323.0,...,NaN,35,2016-10,640001121,씨제이10%포도당주사액 500mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1
9,10DW500,10DW500 10%포도당주사액 500ml (씨제이),10DW500 10%포도당주사액 500ml (씨제이),dextrose(10%),2013-01-24,2999-12-31,원내,1393.0,2780.0,323.0,...,NaN,61,2016-11,640001121,씨제이10%포도당주사액 500mL,씨제이헬스케어(주),Dextrose 10g,2017-02,2017-03,1


In [120]:
use_amount_df[use_amount_df['약품명(한글)'].str.contains('이케이')]['사용량']

4277   NaN
Name: 사용량, dtype: float64